In [ ]:
!pip install langchain==0.1.20 langchain-community==0.0.38 langchain-openai==0.1.7 chromadb pypdf -q

In [ ]:
# Importações básicas
import os

# loader de documentos PDF
from langchain_community.document_loaders import PyPDFLoader

# Divisão de texto em blocos
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Embeddings
from langchain_openai import OpenAIEmbeddings

# Banco vetorial
from langchain_community.vectorstores import Chroma

# LLM
from langchain_openai import ChatOpenAI

# Cadeia RAG
from langchain.chains import RetrievalQA

In [ ]:
# Caminho do PDF com regras .
CAMINHO_PDF = "/content/regras_vendas_ecommerce_beleza.pdf" #ajuste o caminho se necessário

# Carrega o PDF
loader = PyPDFLoader(CAMINHO_PDF)
documents = loader.load()

# Quantidade de páginas carregadas
len(documents)

2

In [ ]:
# Divide os documentos em chunks menores
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

chunks = text_splitter.split_documents(documents)

len(chunks)

7

In [ ]:
# Inicializa embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key="COLOQUEAQUIAAPIKEY" # apagar depois
)

# Cria o banco vetorial
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_regras_vendas_ecommerce_beleza"
)

In [ ]:
# Cria o retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [ ]:
# Inicializa o modelo de linguagem
llm = ChatOpenAI(
    model="gpt-4.1-nano",
    openai_api_key="COLOQUEAQUIAAPIKEY" # apagar
)
# Cria a cadeia RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
# Pergunta de teste
pergunta = "me de 2 regras das vendas"

# Executa a pergunta no agente RAG
resposta = qa_chain(pergunta)

print("Pergunta:")
print(pergunta)

print("\nResposta do Agente:")
print(resposta["result"])

print("\nTrechos utilizados como contexto:\n")
for i, doc in enumerate(resposta["source_documents"], start=1):
    print(f"--- Trecho {i} ---")
    print(f"Fonte: {doc.metadata.get('source', 'Documento desconhecido')}")
    print(f"Página: {doc.metadata.get('page', 'N/A')}\n")
    print("Conteúdo:")
    print(doc.page_content)
    print("\n")

/usr/local/lib/python3.12/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Pergunta:
me de 2 regras das vendas

Resposta do Agente:
Claro! Aqui estão duas regras das vendas:

1. Produtos devem ser devolvidos sem uso e em sua embalagem original para troca ou devolução.
2. Pedidos podem ser cancelados antes do envio, sem custo adicional.

Trechos utilizados como contexto:

--- Trecho 1 ---
Fonte: /content/regras_vendas_ecommerce_beleza.pdf
Página: 0

Conteúdo:
Regras de Vendas – E-commerce de Produtos de Beleza
Este documento estabelece as regras, condições e boas práticas aplicáveis às vendas
realizadas por meio do e-commerce de produtos de beleza, garantindo transparência,
conformidade legal e segurança para clientes e para a empresa.
1. Cadastro e Conta do Cliente
●
O cliente deve fornecer informações verdadeiras, completas e atualizadas.
●
Cada cliente é responsável por manter a confidencialidade de seu login e senha.
●


--- Trecho 2 ---
Fonte: /content/regras_vendas_ecommerce_beleza.pdf
Página: 1

Conteúdo:
6. Trocas e Devoluções
●
O cliente pode solicita